<a href="https://colab.research.google.com/github/vadimvik/tabular_playground_series_jan_2021/blob/main/tabular_playground_series_jan_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#подключим google disk
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install optuna

     |████████████████████████████████| 286kB 4.2MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
     |████████████████████████████████| 163kB 13.5MB/s 
     |████████████████████████████████| 112kB 16.4MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 133kB 16.3MB/s 
     |████████████████████████████████| 481kB 15.3MB/s 
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=afd73b72201b141e1d18af7d6e892308ba0b823882fa237098b89d17903bdf1c
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=bb7dfbec459198c3fb7b8631bc87ba35503266b40876db66e08ecfe3daf14790
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created wheel for pyperclip: filename=pyperclip-1.8.1-cp36-none-any.whl size=11120 sha

In [ ]:
import lightgbm as lgb
import optuna.integration.lightgbm as oplgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv("/content/drive/My Drive/Datasets/tabular-playground-series-jan-2021/train.csv")
df_test = pd.read_csv("/content/drive/My Drive/Datasets/tabular-playground-series-jan-2021/test.csv")
df_sample = pd.read_csv("/content/drive/My Drive/Datasets/tabular-playground-series-jan-2021/sample_submission.csv")

In [ ]:
train_id = df_train["id"]
test_id = df_test["id"]

df_train.drop("id", axis=1, inplace=True)
df_test.drop("id", axis=1, inplace=True)

In [ ]:
df_train

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
0,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
1,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
2,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
3,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
4,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.216974,0.735265,0.648648,0.255387,0.616353,0.345197,0.295718,0.304357,0.314351,0.860504,0.315397,0.247682,0.486542,0.288750,7.385215
299996,0.545799,0.165139,0.220966,0.190053,0.359362,0.386336,0.365767,0.344217,0.466446,0.454581,0.360251,0.360755,0.292535,0.619984,7.242617
299997,0.284401,0.841542,0.957585,0.340383,0.396279,0.330376,0.525687,0.260039,0.378174,0.526925,0.491735,0.516629,0.173521,0.714552,9.592487
299998,0.481900,0.622346,0.540032,0.823118,0.283066,0.434283,0.174342,0.710843,0.358690,0.648272,0.984647,1.001110,0.063956,0.377693,8.207951


In [ ]:
train_id

0              1
1              3
2              4
3              5
4              8
           ...  
299995    499995
299996    499996
299997    499997
299998    499998
299999    499999
Name: id, Length: 300000, dtype: int64

In [ ]:
feature_cols = [c for c in df_train.columns if c != "target"]

In [ ]:
feature_cols

['cont1',
 'cont2',
 'cont3',
 'cont4',
 'cont5',
 'cont6',
 'cont7',
 'cont8',
 'cont9',
 'cont10',
 'cont11',
 'cont12',
 'cont13',
 'cont14']

In [ ]:
train_x = df_train[feature_cols]
train_y = df_train.target
test_x = df_test

In [ ]:
train_x

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411
1,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484
2,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047
3,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528
4,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.216974,0.735265,0.648648,0.255387,0.616353,0.345197,0.295718,0.304357,0.314351,0.860504,0.315397,0.247682,0.486542,0.288750
299996,0.545799,0.165139,0.220966,0.190053,0.359362,0.386336,0.365767,0.344217,0.466446,0.454581,0.360251,0.360755,0.292535,0.619984
299997,0.284401,0.841542,0.957585,0.340383,0.396279,0.330376,0.525687,0.260039,0.378174,0.526925,0.491735,0.516629,0.173521,0.714552
299998,0.481900,0.622346,0.540032,0.823118,0.283066,0.434283,0.174342,0.710843,0.358690,0.648272,0.984647,1.001110,0.063956,0.377693


In [ ]:
train_y

0         7.243043
1         8.203331
2         7.776091
3         6.957716
4         7.951046
            ...   
299995    7.385215
299996    7.242617
299997    9.592487
299998    8.207951
299999    8.890285
Name: target, Length: 300000, dtype: float64

In [ ]:
folds = KFold(n_splits=5, shuffle=True, random_state=2021)

In [ ]:
class FoldsAverageLGBM:
    def __init__(self, folds):
        self.folds = folds
        self.models = []
        
    def fit(self, lgb_params, train_x, train_y):
        oof_preds = np.zeros_like(train_y)
        
        self.train_x = train_x.values
        self.train_y = train_y.values
        
        for tr_idx, va_idx in tqdm(folds.split(train_x)):
            tr_x, va_x = self.train_x[tr_idx], self.train_x[va_idx]
            tr_y, va_y = self.train_y[tr_idx], self.train_y[va_idx]
            
            lgb_train_dataset = lgb.Dataset(tr_x, tr_y)
            lgb_valid_dataset = lgb.Dataset(va_x, va_y)
            model = lgb.train(lgb_params, lgb_train_dataset, valid_sets=[lgb_valid_dataset], verbose_eval=100)
            self.models.append(model)
            
            oof_pred = model.predict(va_x)
            oof_preds[va_idx] = oof_pred
            
        self.oof_preds = oof_preds
        
    def predict(self, test_x):
        preds = []
        for model in tqdm(self.models):
            pred = model.predict(test_x)
            preds.append(pred)
        preds = np.mean(preds, axis=0)
        return preds

In [ ]:
best_lgb_params = {
 'seed': 2121,
 'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'feature_pre_filter': False,
 'lambda_l1': 6.540486456085813,
 'lambda_l2': 0.01548480538099245,
 'num_leaves': 256,
 'feature_fraction': 0.52,
 'bagging_fraction': 0.6161835249194311,
 'bagging_freq': 7,
 'min_child_samples': 20
}
best_lgb_params["learning_rate"] = 0.001
best_lgb_params["early_stopping_round"] = 1000
best_lgb_params["num_iterations"] = 20000

In [ ]:
folds_average_lgbm = FoldsAverageLGBM(folds)

In [ ]:
folds_average_lgbm.fit(best_lgb_params, train_x, train_y)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning:

Found `early_stopping_round` in params. Will use it instead of argument



Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's rmse: 0.730568
[200]	valid_0's rmse: 0.728141
[300]	valid_0's rmse: 0.725931
[400]	valid_0's rmse: 0.723948
[500]	valid_0's rmse: 0.722159
[600]	valid_0's rmse: 0.72055
[700]	valid_0's rmse: 0.719059
[800]	valid_0's rmse: 0.717718
[900]	valid_0's rmse: 0.716509
[1000]	valid_0's rmse: 0.715394
[1100]	valid_0's rmse: 0.714377
[1200]	valid_0's rmse: 0.713428
[1300]	valid_0's rmse: 0.712551
[1400]	valid_0's rmse: 0.711761
[1500]	valid_0's rmse: 0.711007
[1600]	valid_0's rmse: 0.710317
[1700]	valid_0's rmse: 0.709676
[1800]	valid_0's rmse: 0.709083
[1900]	valid_0's rmse: 0.708529
[2000]	valid_0's rmse: 0.708009
[2100]	valid_0's rmse: 0.707524
[2200]	valid_0's rmse: 0.707074
[2300]	valid_0's rmse: 0.706633
[2400]	valid_0's rmse: 0.706224
[2500]	valid_0's rmse: 0.705835
[2600]	valid_0's rmse: 0.705467
[2700]	valid_0's rmse: 0.705123
[2800]	valid_0's rmse: 0.704789
[2900]	valid_0's rmse: 0.704485
[3000]	valid_0's r

In [ ]:
np.sqrt(mean_squared_error(df_train.target, folds_average_lgbm.oof_preds))

0.6955419749557664

In [ ]:
y_pred = folds_average_lgbm.predict(test_x)

In [ ]:
sub = df_sample.copy()
sub["target"] = y_pred

sub.to_csv("/content/drive/My Drive/Datasets/tabular-playground-series-jan-2021/submission_lgbm_1.csv", index=False)

sub.head()

,id,target
0,0,7.930961
1,2,7.880882
2,6,7.935323
3,7,8.278395
4,10,8.093543
